In [1153]:
import ortools
from ortools.init.python import init
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# 0. Data import and solver set up

In [1154]:
distances = pd.read_excel('distances.xlsx', index_col=0, skiprows=1, usecols=range(1, 6))
print("Shape of distances dataset : ",distances.shape)
display(distances.head())
index_values = pd.read_excel('indexValues.xlsx', header=None, names=["indexValue"], usecols=[1])
print("Shape of indexValues dataset : ",index_values.shape)
display(index_values.head())
x_origin = pd.read_excel('x_origin.xlsx', index_col=0,skiprows=1, usecols=range(1, 6))
print("Shape of x' dataset : ",x_origin.shape)
display(x_origin.head())

Shape of distances dataset :  (22, 4)


,1,2,3,4
1,16.16,24.08,24.32,21.12
2,19.00,26.47,27.24,17.33
3,25.29,32.49,33.42,12.25
4,0.00,7.93,8.31,36.12
5,3.07,6.44,7.56,37.37


Shape of indexValues dataset :  (22, 1)


,indexValue
0,0.1609
1,0.1164
2,0.1026
3,0.1516
4,0.0939


Shape of x' dataset :  (22, 4)


,1,2,3,4
x'ij,,,,
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,1,0,0,0
5,1,0,0,0


# 1. Variables

In [1155]:
def set_variables(index_values,distances):
    num_bricks = len(index_values)
    num_sr = len(distances.columns)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    assignment = [[solver.IntVar(0, 1, f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, assignment, num_bricks, num_sr

# 2. Constraints

In [1156]:

def constraint_assignment_bricks(solver, assignment, num_bricks, num_sr):
    # add a constraint on the columns ?
    for i in range(num_bricks):
        solver.Add(sum(assignment[i][j] for j in range(num_sr)) == 1)

def constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval):
    for j in range(num_sr):
        total_weight = solver.Sum(assignment[i][j] * index_values.iloc[i,0] for i in range(num_bricks))
        solver.Add(total_weight >= min(condition_interval))
        solver.Add(total_weight <= max(condition_interval))

def apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

# 3. Objective functions

In [1157]:
# Objective function: minimize weighted distance + weight-based disruption penalty
def get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant):
    objective = solver.Objective()
    for i in range(num_bricks):
        for j in range(num_sr):
            # Weighted distance cost
            distance_cost = distances.iloc[i,j]
            
            # Disruption penalty if reassigned to a different SR
            if x_origin.iloc[i,j] == 0:
                disruption_penalty = penalty_constant 
            else:
                disruption_penalty = 0

            # Total cost for assigning brick i to SR j
            total_cost = distance_cost + disruption_penalty
            objective.SetCoefficient(assignment[i][j], total_cost)

    objective.SetMinimization()

    return objective

# 4. Solve

In [1158]:
def get_stats(assignment, num_sr, num_bricks, objective):
    sr_weights = [0.0] * num_sr
    sr_distances = [0.0] * num_sr
    for j in range(num_sr):
        sr_bricks = []
        for i in range(num_bricks):
            if assignment[i][j].solution_value() > 0.0:
                sr_weights[j] += index_values.iloc[i,0] * assignment[i][j].solution_value()
                sr_distances[j] += distances.iloc[i,j] 
                sr_bricks.append(i)
        print(f"SR {j+1} total weight is {round(sr_weights[j],2)} / total distances is {round(sr_distances[j],2)} / assigned bricks are {sr_bricks}")
    # Calculate the sum of (sum of weights per SR * sum of distances per SR)
    total_distances = sum(sr_distances)
    total_weighted_distance = sum(sr_weights[j] * sr_distances[j] for j in range(num_sr))
    print('Sum of distances:', total_distances)
    print('Sum of weighted distances:', total_weighted_distance)
    print('Optimal Objective Value:', objective.Value())
    
def solve_print(solver, assignment, objective, num_bricks, num_sr):
    # Solve the problem
    status = solver.Solve()
    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        get_stats(assignment, num_sr, num_bricks, objective)
        
    else:
        print('No optimal solution found.')


# 5. Get Results

In [1159]:
def get_results(distances,index_values, condition_interval, penalty_constant):
    solver, assignment, num_bricks, num_sr = set_variables(index_values,distances)
    apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval)
    objective = get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant)
    solve_print(solver, assignment, objective, num_bricks, num_sr)

In [1160]:
condition_interval = [0.8,1.2]
penalty_constant = 0
get_results(distances, index_values, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.04 / total distances is 64.37 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19]
SR 2 total weight is 1.04 / total distances is 7.53 / assigned bricks are [10, 12, 13, 17]
SR 3 total weight is 1.11 / total distances is 6.57 / assigned bricks are [9, 14, 15, 16]
SR 4 total weight is 0.8 / total distances is 76.13 / assigned bricks are [0, 1, 2, 20, 21]
Sum of distances: 154.6
Sum of weighted distances: 143.09896
Optimal Objective Value: 154.60000000000002


# 6. Questions

### 6.1. Do the obtained solutions have properties that are worth bringing to the attention of the decision maker?

- **Total Weighted Distance**: The solution has a total weighted distance of approximately 143.1. This metric helps the decision maker understand the overall efficiency and cost of the current SR (Sales Representatives) allocation.

- **Weight Balance**: Each SR has a total weight within the constraint interval (between 0.8 and 1.2), indicating a balanced workload. SRs 1, 2, and 3 are near the upper limit, while SR 4 is close to the minimum requirement.

- **Distance Distribution**: SRs 1 and 4 have significantly higher total distances (64.37 and 76.13 respectively) compared to SRs 2 and 3 (7.53 and 6.57). This suggests that SRs 1 and 4 cover more geographically dispersed bricks, which may imply higher travel times or costs.

- **Assignment of Bricks**: SRs are assigned a varied number of bricks, with SR 1 covering nine bricks while SR 2, 3, and 4 cover fewer. This indicates that some SRs may face higher workload concentration in certain areas, which could impact operational efficiency.

- **Assignment distruption**: good metric to evaluate how much changes we have between each assignment

### 6.2. Discuss the properties of the current solution in comparison to other solutions obtained.

- **Workload Distribution**: Compared to other solutions where the workload might not be as evenly distributed, this solution appears to have achieved a balance across SRs, ensuring that no SR is overwhelmed by an excessive workload.

- **Distance Optimization**: By minimizing the weighted distance, this solution achieves a relatively low-cost allocation. However, if there are alternate solutions with slightly higher costs but more balanced distances across SRs, these could be considered for improved efficiency.

- **Disruption Penalty**: This solution minimizes reassignment of bricks compared to the origin (x_origin) due to the disruption penalty applied. This minimizes costs associated with changing SR responsibilities.

### 6.3. Varying workload is currently an objective taken into account in the form of constraints. We can get different sets of effective solutions and tradeoffs with other goals by tightening and relaxing this constraint. Try at least one interval and discuss the result (you can use [0.9, 1.1])

In [1161]:
condition_interval = [0.9,1.1]
penalty_constant = 10
get_results(distances, index_values, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.06 / total distances is 30.43 / assigned bricks are [3, 4, 5, 6, 7, 14, 18]
SR 2 total weight is 1.08 / total distances is 28.8 / assigned bricks are [10, 11, 12, 13]
SR 3 total weight is 0.96 / total distances is 14.4 / assigned bricks are [8, 9, 15, 16, 17]
SR 4 total weight is 0.9 / total distances is 99.34 / assigned bricks are [0, 1, 2, 19, 20, 21]
Sum of distances: 172.97
Sum of weighted distances: 166.79111999999998
Optimal Objective Value: 192.97


SRs that were previously assigned workloads close to the upper and lower limits (like SR 4) will likely be forced to take on additional bricks or reduce their assigned bricks, shifting assignments more uniformly across SRs.
This tighter constraint increases the total weighted distance since the model has fewer assignment options to minimize cost. If the workload becomes more balanced, some SRs may cover fewer distances overall, while others may need to take on more, resulting in a trade-off between cost and balanced distribution.

### 6.4. How to model the case for partially assigning bricks (i.e. assign a brick to multiple SR)? Implement this and compare the results.

In [1162]:
  
def set_variables(index_values,distances):
    num_bricks = len(index_values)
    num_sr = len(distances.columns)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    partial_assignment = [[solver.NumVar(0, 1, f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, partial_assignment, num_bricks, num_sr #, assignment

In [1163]:
condition_interval = [0.8,1.2]
penalty_constant = 100
get_results(distances, index_values, condition_interval,penalty_constant)
print('-'*100)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 0.95 / total distances is 19.3 / assigned bricks are [3, 4, 5, 6, 7, 14]
SR 2 total weight is 1.2 / total distances is 33.32 / assigned bricks are [9, 10, 11, 12, 13]
SR 3 total weight is 0.84 / total distances is 12.12 / assigned bricks are [8, 13, 15, 16, 17]
SR 4 total weight is 1.01 / total distances is 124.74 / assigned bricks are [0, 1, 2, 18, 19, 20, 21]
Sum of distances: 189.48000000000002
Sum of weighted distances: 194.13184200000006
Optimal Objective Value: 204.59850311850312
----------------------------------------------------------------------------------------------------


### 6.5.  If the demand increases uniformly in all bricks (for example + 20%, it may be necessary to hire a new sales representative. There is the question of where to locate his office (center brig)

In [1164]:
x_origin['5'] = [0 for i in range(22)]

In [1165]:
brick_to_brick = pd.read_excel("distances.xlsx", sheet_name=["brick-brick"],  index_col=0, skiprows=1, usecols=range(1, 24))
brick_to_brick = brick_to_brick['brick-brick']

In [1166]:
new_distances = brick_to_brick.to_numpy()

In [1167]:
M = 100000
epsilon = 1e-6

In [1168]:
def set_variables(index_values,distances, num_sr):
    num_bricks = len(index_values)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    assignment = [[solver.NumVar(0, 1, f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]
    # use it for general cas
    office_assignment = [solver.IntVar(0, 1, f'office_assignment_{i}') for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, assignment, num_bricks, office_assignment

In [1169]:
def constraint_assignment_center_office(solver, office_assignment, num_sr, already_assigned_offices_indices):
    # only one office per sr
    solver.Add(sum(office_assignment) == num_sr)
    for i in already_assigned_offices_indices:
        solver.Add(office_assignment[i] == 1)

def apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval, already_assigned_offices_indices):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

    # 3. Assignment of offices
    constraint_assignment_center_office(solver, office_assignment, num_sr, already_assigned_offices_indices)

In [1170]:
# Objective function: minimize weighted distance + weight-based disruption penalty + weighted office assignment
def get_objective_function(solver, assignment, office_assignment, x_origin, num_bricks, num_sr, penalty_constant, distances):
    objective = solver.Objective()
    for j in range(num_sr):                
        for center in range(num_bricks):
            for i in range(num_bricks): 
                # Weighted distance cost
                distance_cost = distances[center][i]
                
                # Disruption penalty if reassigned to a different SR
                if x_origin.iloc[i,j] == 0:
                    disruption_penalty = penalty_constant 
                else:
                    disruption_penalty = 0

                # Total cost for assigning brick i to SR j
                total_cost = distance_cost + disruption_penalty
                objective.SetCoefficient(assignment[i][j], total_cost)
                objective.SetCoefficient(office_assignment[center], total_cost)

    objective.SetMinimization()

    return objective

In [1171]:

def get_results(distances,index_values, num_sr, condition_interval, penalty_constant, already_assigned_offices_indices):
    solver, assignment, num_bricks, office_assignment = set_variables(index_values,distances, num_sr)
    apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval, already_assigned_offices_indices)
    objective = get_objective_function(solver, assignment, office_assignment, x_origin, num_bricks, num_sr, penalty_constant, distances)
    solve_print(solver, assignment, office_assignment, objective, num_bricks, num_sr, distances)

In [1292]:
def get_stats(assignment, office_assignment, num_sr, num_bricks, objective, distances):
    sr_weights = []
    sr_distances = []

    centers = []
    all_assignments = []
    ## calculate center
    # get assignment for sr
    # get distances from all centers 
    for sr in range(num_sr):
        list_asignment, candidates_for_assign, assignment_values = [], [], []
        for b in range(num_bricks):
            if assignment[b][sr].solution_value() > 0.0:
                list_asignment.append(b)
                assignment_values.append(assignment[b][sr].solution_value())
                if office_assignment[b].solution_value() == 1 :
                    candidates_for_assign.append(b)
            
            if office_assignment[b].solution_value() == 1 and sr == 0:
                centers.append(b)

        all_assignments.append((list_asignment, candidates_for_assign, assignment_values))
    # center in assignment
    def attribute_center(all_assignments, cleaned_assignments, centers, stop=5):
        if stop == 0 or all_assignments == []:
            return cleaned_assignments

        to_delete = [all_assignments[i] for i in range(len(all_assignments)) if len(all_assignments[i][1]) == 1] 
        center_assigned = [i[1][0] for i in to_delete]
        centers = [i for i in centers if i not in center_assigned]
        cleaned_assignments.extend(to_delete)
        all_assignments =  [all_assignments[i]  for i in range(len(all_assignments)) if len(all_assignments[i][1]) != 1]


        for i in range(len(to_delete)):
            for j in range(len(all_assignments)):
                if to_delete[i][1][0] in all_assignments[j][1]:
                    all_assignments[j][1].remove(to_delete[i][1][0])
        
        return attribute_center(all_assignments, cleaned_assignments, centers, stop = stop - 1)
    # print(centers) 
    # print(all_assignments)
    center_assignments = attribute_center(all_assignments, [], centers)
    for center in range(num_sr):
        sr_weight = 0
        sr_distance = 0
        for i in range(len(center_assignments[center][0])):
            # print(f"assignment: {center_assignments[center][2][i]}")
            # print(f"center: {center_assignments[center][1]}")
            sr_weight += index_values.iloc[center_assignments[center][0][i], 0] * center_assignments[center][2][i]
            sr_distance += distances[center_assignments[center][1][0]][center_assignments[center][0][i]] 
        sr_weights.append(sr_weight)
        sr_distances.append(sr_distance)
        print(f"SR {center+1} total weight is {round(sr_weights[center],2)} / total distances is {round(sr_distances[center],2)} / assigned bricks are {(center_assignments[center][0])} with center brick {center_assignments[center][1][0]}")

    # Calculate the sum of (sum of weights per SR * sum of distances per SR)
    total_distances = sum(sr_distances)
    total_weighted_distance = sum(sr_weights[j] * sr_distances[j] for j in range(num_sr))
    print('Sum of distances:', total_distances)
    print('Sum of weighted distances:', total_weighted_distance)
    print('Optimal Objective Value:', objective.Value())
    
def solve_print(solver, assignment, office_assignment, objective, num_bricks, num_sr, distances):
    # Solve the problem
    status = solver.Solve()
    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        get_stats(assignment, office_assignment, num_sr, num_bricks, objective, distances)
        
    else:
        print('No optimal solution found.')


In [1173]:
index_values = index_values * 1.2

In [1174]:
num_sr = 5
already_assigned_offices_indices = np.where(distances == 0)[0]
condition_interval = [0.8,1.2]
penalty_constant = 100
get_results(new_distances, index_values, num_sr, condition_interval,penalty_constant, already_assigned_offices_indices)

Number of brics :  22
Number of SR :  5
Solution found:
SR 1 total weight is 1.14 / total distances is 19.3 / assigned bricks are [3, 4, 5, 6, 7, 14] with center brick 3
SR 2 total weight is 0.81 / total distances is 33.32 / assigned bricks are [9, 10, 11, 12, 13] with center brick 13
SR 3 total weight is 0.85 / total distances is 9.99 / assigned bricks are [8, 15, 16, 17] with center brick 15
SR 4 total weight is 0.8 / total distances is 43.75 / assigned bricks are [13, 21] with center brick 21
SR 5 total weight is 1.2 / total distances is 72.89 / assigned bricks are [0, 1, 2, 18, 19, 20, 21] with center brick 2
Sum of distances: 179.2474570907326
Sum of weighted distances: 180.03563177965395
Optimal Objective Value: 1463.9829059574108


### 6.6.  The location of the "center bricks" (SR offices) has a significant impact on the distance traveled by the SRs. An important question is to generalize the model so as to allow a modification of the "center bricks"; this requires considering additional binary variables. (It should be noted that some of the bricks are not good candidates as a "center brig", which makes it possible to reduce the number of variables).

In [1178]:
def constraint_assignment_center_office(solver, office_assignment, assignment, num_sr, num_bricks):
    # only one office per sr
    solver.Add(sum(office_assignment) == num_sr)
    # for j in range(num_sr):
    #     solver.Add(sum(assignment[i][j] * office_assignment[i] for i in range(num_bricks)) > 0)

def apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

    # 3. Assignment of offices
    constraint_assignment_center_office(solver, office_assignment, assignment, num_sr, num_bricks)

In [1179]:
def get_results(distances,index_values, num_sr, condition_interval, penalty_constant):
    solver, assignment_value, num_bricks, office_assignment, assignment = set_variables(index_values,distances, num_sr)
    apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval, already_assigned_offices_indices, assignment_value)
    objective = get_objective_function(solver, assignment, office_assignment, x_origin, num_bricks, num_sr, penalty_constant, distances)
    solve_print(solver, assignment_value, office_assignment, objective, num_bricks, num_sr, distances)

In [1180]:
num_sr = 5
condition_interval = [0.8,1.2]
penalty_constant = 100
get_results(new_distances, index_values, num_sr, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  5


ValueError: not enough values to unpack (expected 5, got 4)

To generalize the model by allowing SR office relocations, we have to:

- **Add Binary Variables**: Introduce binary variables to indicate the selection of certain bricks as new SR centers. Not all bricks are candidates, so restrict the variable to a subset of bricks that are potential centers.

- **Objective Function Adjustment**: Incorporate the distances from each brick to the selected SR centers, modifying the objective to minimize both weighted distance and potential disruption penalties due to reassignment. With this the model can adapt dynamically to changes in demand and geography by choosing optimal SR office locations, potentially lowering travel distances and improving response times.

Each modification builds on the initial model to address different logistical scenarios, improving operational efficiency under changing conditions.

### 6.7  How to incorporate in the model the SRs' preferences for their area?

In [1293]:
num_sr = 5
already_assigned_offices_indices = np.where(distances == 0)[0]
already_assigned_offices_indices = np.delete(already_assigned_offices_indices, 3)
print(already_assigned_offices_indices)

[ 3 13 15]


In [1294]:
office_preferences = [[0.2, 0.3, 0.4, 0, 0, 0, 0.5, 0.6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0] * 22, [0] * 22, [0] * 22, [0] * 22] # format len num_bricks
office_preferences[1][2] = 0.8
office_preferences[1][18] = 0.5
office_preferences[1][19] = 0.5
office_preferences[1][20] = 0.5
office_preferences[1][21] = 0.3

In [1295]:
def set_variables(index_values,distances, num_sr):
    num_bricks = len(index_values)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    assignment = [[solver.NumVar(0, 1, f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]
    # use it for general cas
    office_assignment = [solver.IntVar(0, 1, f'office_assignment_{i}') for i in range(num_bricks)]
    office_assignment_final = [solver.IntVar(0, 1, f'office_assignment_w_preference_{i}') for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, assignment, num_bricks, office_assignment, office_assignment_final

In [1296]:
def constraint_assignment_center_office(solver, office_assignment, assignment, num_sr, num_bricks, already_assigned_offices_indices, office_assignment_final, office_preferences):
    # only one office per sr
    solver.Add(sum(office_assignment) == num_sr)
    # for j in range(num_sr):
    #     total_weight = solver.Sum(assignment[i][j] * index_values.iloc[i,0] for i in range(num_bricks))
    for i in range(num_bricks):
        for j in range(num_sr):
            # Linearization of product for binary variables
            solver.Add(office_assignment_final[i] <= office_assignment[i])
            solver.Add(office_assignment_final[i] <= office_preferences[j][i])
            solver.Add(office_assignment_final[i] >= office_assignment[i] + office_preferences[j][i] - 2)


            # solver.Add(sum(office_assignment_final[j]) == sum(office_preferences[j][i] * office_assignment[i]))
    # solver.Add(sum(office_assignment_final[j][i] for j in range(num_sr) for i in range(num_bricks)) == sum(office_assignment[i] * office_preferences[j][i] for j in range(num_sr) for i in range(num_bricks)))
    for i in already_assigned_offices_indices:
        solver.Add(office_assignment[i] == 1)
    # for j in range(num_sr):
    #     solver.Add(sum(assignment[i][j] * office_assignment[i] for i in range(num_bricks)) > 0)

def apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval, already_assigned_offices_indices, office_assignment_final, office_preferences):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

    # 3. Assignment of offices
    constraint_assignment_center_office(solver, office_assignment, assignment, num_sr, num_bricks, already_assigned_offices_indices, office_assignment_final, office_preferences)

In [1297]:
# Objective function: minimize weighted distance + weight-based disruption penalty + weighted office assignment
def get_objective_function(solver, assignment, office_assignment, x_origin, num_bricks, num_sr, penalty_constant, distances, office_preferences):
    objective = solver.Objective()
    for j in range(num_sr):                
        for center in range(num_bricks):
            for i in range(num_bricks): 
                # Weighted distance cost
                distance_cost = distances[center][i]
                
                # Disruption penalty if reassigned to a different SR
                if x_origin.iloc[i,j] == 0:
                    disruption_penalty = penalty_constant 
                else:
                    disruption_penalty = 0

                # Total cost for assigning brick i to SR j
                total_cost = distance_cost + disruption_penalty
                objective.SetCoefficient(assignment[i][j], total_cost)
                objective.SetCoefficient(office_assignment[center], total_cost)

    objective.SetMinimization()

    return objective

In [1298]:
def get_results(distances,index_values, num_sr, condition_interval, penalty_constant, office_preferences):
    solver, assignment, num_bricks, office_assignment, office_assignment_final = set_variables(index_values,distances, num_sr)
    apply_constraints(solver, assignment, office_assignment, index_values, num_bricks, num_sr, condition_interval, already_assigned_offices_indices, office_assignment_final, office_preferences)
    objective = get_objective_function(solver, assignment, office_assignment, x_origin, num_bricks, num_sr, penalty_constant, distances, office_preferences)
    solve_print(solver, assignment, office_assignment, objective, num_bricks, num_sr, distances)

In [1299]:
num_sr = 5
condition_interval = [0.8,1.2]
penalty_constant = 100
get_results(new_distances, index_values, num_sr, condition_interval,penalty_constant, office_preferences)

Number of brics :  22
Number of SR :  5
Solution found:
SR 1 total weight is 1.14 / total distances is 19.3 / assigned bricks are [3, 4, 5, 6, 7, 14] with center brick 3
SR 2 total weight is 0.81 / total distances is 33.32 / assigned bricks are [9, 10, 11, 12, 13] with center brick 13
SR 3 total weight is 0.85 / total distances is 9.99 / assigned bricks are [8, 15, 16, 17] with center brick 15
SR 4 total weight is 0.8 / total distances is 43.75 / assigned bricks are [13, 21] with center brick 21
SR 5 total weight is 1.2 / total distances is 72.89 / assigned bricks are [0, 1, 2, 18, 19, 20, 21] with center brick 2
Sum of distances: 179.2474570907326
Sum of weighted distances: 180.03563177965395
Optimal Objective Value: 1463.9829059574113


To incorporate SRs' area preferences, we could add:

- **Preference Penalty in Objective Function**: Add a penalty to the objective for assignments that do not match SRs’ preferences, encouraging alignment without imposing hard constraints.

- **Weighted Assignments**: Assign weights to bricks based on SRs' preferences, prioritizing preferred areas.